In [ ]:
!pip install torch -U
!pip install transformers
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import transformers
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd

import accelerate
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

### GPU 설정하기,

device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
print("device : " , device)

device :  cuda:0


## 1. 토크나이저 및 모델 불러오기

In [ ]:
Model_name = 'beomi/KcELECTRA-base-v2022'
tokenizer = AutoTokenizer.from_pretrained(Model_name)

## 2. 데이터 전처리

In [ ]:
data1 = pd.read_excel("/content/drive/MyDrive/modeling/data/word_in_언감O.xlsx", index_col = 0)
data2 = pd.read_excel("/content/drive/MyDrive/modeling/data/word_out_언감O.xlsx", index_col = 0)
data3 = pd.read_excel("/content/drive/MyDrive/modeling/data/wrong.xlsx", index_col = 0)

In [ ]:
data = pd.concat([data1, data2, data3], axis = 0)

In [ ]:
data.index = range(len(data))

In [ ]:
data

,sentence,장애및병력,성과가족,사회적신분,출신,욕설,기타,언어감수성_X,언어감수성_O
0,오늘 날씨 대박 좋다,0,0,0,0,0,0,0,1
1,윤석열 역도를 몰아내자,0,0,0,0,0,0,0,1
2,근데 양성애자가 성소수자임,0,0,0,0,0,0,0,1
3,섹추털은 왜 구불구불함,0,0,0,0,0,0,0,1
4,내가 한달동안 자폐아 돌보는 봉사해봐서 아는데,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
420970,년 이후 다문화 가정폭력 검거 건수가 무려 건에 달하고 있다,0,0,0,1,0,0,1,0
420971,보조금을 눈먼 돈 쌈짓돈이라 생각하니 그런 행태를 보이는 것이다,1,0,0,0,0,0,1,0
420972,그러나 의도가 어떻든 간에 지금 한국 사회에서 가장 대책이 시급한 문제로 저출산을 ...,0,1,0,0,0,0,1,0
420973,나라가 망하는 방법에는 여러 가지가 있지만 지금 출산율을 이어가는 것은 가장 확실히...,0,1,0,0,0,0,1,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['sentence'], data['언어감수성_X'], test_size=0.2, shuffle = True)

In [ ]:
tokenized_train_sentence = tokenizer(
    list(X_train),
    return_tensors = 'pt',
    max_length = 128,
    padding = True,
    truncation = True,
    add_special_tokens = True)

In [ ]:
print(tokenized_train_sentence[0].tokens)

['[CLS]', '밥먹', '##는거', '밀리', '##고', '누구', '오', '##기', '기다리고', '이러면', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[P

In [ ]:
tokenized_test_sentence = tokenizer(
    list(X_test),
    return_tensors = 'pt',
    max_length = 128,
    padding = True,
    truncation = True,
    add_special_tokens = True)

## 3. 데이터 변환

In [ ]:
class CurseDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

## 4. 모델 학습

In [ ]:
train_label = y_train.values
test_label = y_test.values

train_dataset = CurseDataset(tokenized_train_sentence, train_label)
test_dataset = CurseDataset(tokenized_test_sentence, test_label)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(Model_name, num_labels = 2)
model.to(device)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
import accelerate

In [ ]:
training = TrainingArguments(
    output_dir = './',
    num_train_epochs = 8,
    per_device_train_batch_size = 32 ,
    per_device_eval_batch_size = 64 ,
    logging_dir = './logs' ,
    logging_steps = 500 ,
    save_total_limit = 2 ,
)

In [ ]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision , recall, f1, _ = precision_recall_fscore_support(labels, preds, average = 'binary')
  acc = accuracy_score(labels, preds)
  return {
      'accuracy ' : acc,
      'f1': f1,
      'precision' : precision,
      'recall' : recall
  }

In [ ]:
trainer = Trainer(
    model = model,
    args = training,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics = compute_metrics,
)

In [ ]:
trainer.train()

<ipython-input-16-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,0.252200
1000,0.107500
1500,0.080500
2000,0.068000
2500,0.066000
3000,0.064300
3500,0.052700
4000,0.056800
4500,0.045000
5000,0.046400


<ipython-input-16-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-16-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-16-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-16-d3da3f7a2d94>:7: UserWarning: To copy construct from a 

In [ ]:
 trainer.evaluate(eval_dataset = test_dataset)

In [ ]:
#여기까지 실행 후, 맨 마지막줄 실행할 것

In [ ]:
def predict(sent):
  model.eval()

  tokenized_sent = tokenizer(
      sent,
      return_tensors = 'pt',
      truncation = True,
      add_special_tokens = True,
      max_length = 128
  )

  tokenized_sent.to(device)

  with torch.no_grad():
    outputs = model(
        input_ids = tokenized_sent['input_ids'],
        attention_mask = tokenized_sent['attention_mask'],
        token_type_ids = tokenized_sent['token_type_ids']
    )

  logits = outputs[0]
  logits = logits.detach().cpu()
  result = logits.argmax(-1)

  if result == 0:
    result = '언어감수성이 낮습니다'
  elif result == 1:
    result = '언어감수성이 높습니다'
  return result

In [ ]:
predict("너 오른손잡이야?")

NameError: ignored

In [ ]:
model.save_pretrained('directory_path')
tokenizer.save_pretrained('directory_path')

('directory_path/tokenizer_config.json',
 'directory_path/special_tokens_map.json',
 'directory_path/tokenizer.json')

In [ ]:
from google.colab import files

# 저장된 폴더를 zip 파일로 압축
!zip -r /content/directory_path.zip /content/directory_path

In [ ]:
# 모델 저장 경로 설정
save_path = "/content/drive/MyDrive/modeling/result"

# 모델과 토크나이저 저장
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
